<a href="https://colab.research.google.com/github/Corosso/ValoAI/blob/main/Project_ValoAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Get the data 
acs = pd.read_csv("teams.csv")
map_win_rate = pd.read_csv("map_win_rate.csv")
previous_matches = pd.read_csv("previous_matches.csv")
matchup = pd.read_csv("enfrentamiento.csv")

In [2]:
# Add a new column to store the team name
acs['team'] = acs['team']

# Split the ACS values into separate columns
acs_s = acs['acs'].str.split(',', expand=True).astype(float)
acs_s.columns = ['acs_1', 'acs_2', 'acs_3', 'acs_4', 'acs_5']

# Concatenate the team name with the ACS values
acs_merged = pd.concat([acs['team'], acs_s], axis=1)



In [3]:
# Remove the number in parentheses from the map name
map_win_rate['map'] = map_win_rate['map'].str.replace(r'\(\d+\)', '')
# Merge ACS and map win rate
data = pd.merge(acs_merged, map_win_rate, on="team")
# Merge previous matches with acs and win rate
data = pd.merge(data, previous_matches, left_on="team", right_on="team_A")
# Convert team names to numeric values using label encoder
encoder = LabelEncoder()
data['team'] = encoder.fit_transform(data['team'])
# Convert map names to numeric values using label encoder
encoder = LabelEncoder()
data['map'] = encoder.fit_transform(data['map'])
# Convert team_A and team_B names to numeric values using label encoder
encoder = LabelEncoder()
data['team_A'] = encoder.fit_transform(data['team_A'])
data['team_B'] = encoder.fit_transform(data['team_B'])
# Reeplace ("-") for NaN
data['win_rate'] = data['win_rate'].replace('-', float('nan'))
# Convert win rate to float
data['win_rate'] = data['win_rate'].str.rstrip('%').astype(float) / 100
# Reeplace NaN for zeros
data = data.fillna(0)

<ipython-input-3-3fddf068fb58>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  map_win_rate['map'] = map_win_rate['map'].str.replace(r'\(\d+\)', '')


In [4]:
data

,team,acs_1,acs_2,acs_3,acs_4,acs_5,map,win_rate,team_A,team_B,winner
0,8,186.7,251.5,223.0,220.5,125.0,1,0.68,8,6,1
1,8,186.7,251.5,223.0,220.5,125.0,1,0.68,8,4,0
2,8,186.7,251.5,223.0,220.5,125.0,4,0.81,8,6,1
3,8,186.7,251.5,223.0,220.5,125.0,4,0.81,8,4,0
4,8,186.7,251.5,223.0,220.5,125.0,8,0.75,8,6,1
...,...,...,...,...,...,...,...,...,...,...,...
121,7,250.7,252.7,223.0,186.7,160.0,5,0.64,7,0,1
122,7,250.7,252.7,223.0,186.7,160.0,2,0.71,7,0,1
123,7,250.7,252.7,223.0,186.7,160.0,3,0.57,7,0,1
124,7,250.7,252.7,223.0,186.7,160.0,7,1.00,7,0,1


In [12]:
# Select the input variables (ACS and Map Win Rate)
cols = ['team', 'acs_1', 'acs_2', 'acs_3', 'acs_4', 'acs_5', 'map', 'win_rate']
X = data[cols]
# Select the output variable (winner)
y = data["winner"]

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Create a logistic regression model
model = LogisticRegression(max_iter=10000)

# Train the model with the training data
model.fit(X_train, y_train)

# Evaluate model accuracy with test data
precision = model.score(X_test, y_test)
print("Model precission: ", precision)

Model precission:  0.6538461538461539


In [26]:
# Make predictions on new data

# Which teams are going to play
team_A='Natus Vincere'
team_B='Team Heretics'
t={'team_A':[team_A],'team_B':[team_B]}
team = pd.DataFrame(data=t)
team['team_A'] = encoder.fit_transform(team['team_A'])
team['team_B'] = encoder.fit_transform(team['team_B'])
data_teams = pd.merge(team,data, left_on="team_A", right_on="team")
data_teams = pd.merge(team,data, left_on="team_B", right_on="team")



In [28]:
# Get the probabilities
probs = model.predict_proba(X_teams)

# Get the winner
if probs[0][1] > probs[0][0]:
    winner = team_B
    prob_ganador = probs[0][1]
else:
    winner = team_A
    prob_winner = probs[0][0]

# Print probs and winner
print("El ganador es:", winner)
print("Probabilidad de", team_A, ":", probs[0][0])
print("Probabilidad de", team_B, ":", probs[0][1])
print("Probabilidad del ganador:", prob_winner)

El ganador es: Natus Vincere
Probabilidad de Natus Vincere : 0.9761323181442487
Probabilidad de Team Heretics : 0.023867681855751294
Probabilidad del ganador: 0.9761323181442487
